In [11]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

# Define paths
data_dir = 'brain_tumor_dataset'
categories = ['no_tumor', 'glioma_tumor', 'meningioma_tumor', 'pituitary_tumor']

# Custom Dataset Class
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, categories, transform=None):
        self.root_dir = root_dir
        self.categories = categories
        self.transform = transform
        self.images = []
        self.labels = []

        # Populate image paths and labels
        for category in categories:
            path = os.path.join(root_dir, category)
            class_num = categories.index(category)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                self.images.append(img_path)
                self.labels.append(class_num)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert('RGB')  # Ensure images are in RGB format

        if self.transform:
            img = self.transform(img)

        return img, label

In [12]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (common size for CNNs)
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet normalization
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
# Paths to training and testing folders
train_dir = os.path.join(data_dir, 'Training')
test_dir = os.path.join(data_dir, 'Testing')

# Create datasets
train_dataset = BrainTumorDataset(train_dir, categories, transform=transform)
test_dataset = BrainTumorDataset(test_dir, categories, transform=transform)

In [13]:
from torch.utils.data import DataLoader

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load pre-trained ResNet50 model
model = models.resnet50(weights='IMAGENET1K_V1')

# Replace the final fully connected layer
num_classes = len(categories)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [16]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=20):
    model.train()
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

        # Evaluate on test set
        test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

        # Save the best model
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            torch.save(model.state_dict(), 'best_model.pth')

def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(test_loader)
    epoch_accuracy = correct / total
    return epoch_loss, epoch_accuracy

# Train the model
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=20)

Epoch [1/20], Loss: 0.3431, Accuracy: 0.8784
Test Loss: 1.1382, Test Accuracy: 0.7538
Epoch [2/20], Loss: 0.4112, Accuracy: 0.8578
Test Loss: 1.2019, Test Accuracy: 0.7081
Epoch [3/20], Loss: 0.2119, Accuracy: 0.9289
Test Loss: 1.2673, Test Accuracy: 0.7386
Epoch [4/20], Loss: 0.1153, Accuracy: 0.9589
Test Loss: 1.6355, Test Accuracy: 0.7030
Epoch [5/20], Loss: 0.0257, Accuracy: 0.9906
Test Loss: 1.6294, Test Accuracy: 0.7893
Epoch [6/20], Loss: 0.0984, Accuracy: 0.9707
Test Loss: 1.4393, Test Accuracy: 0.7614
Epoch [7/20], Loss: 0.0331, Accuracy: 0.9892
Test Loss: 2.5757, Test Accuracy: 0.6548
Epoch [8/20], Loss: 0.0441, Accuracy: 0.9868
Test Loss: 1.2001, Test Accuracy: 0.7944
Epoch [9/20], Loss: 0.0160, Accuracy: 0.9941
Test Loss: 2.5970, Test Accuracy: 0.7792
Epoch [10/20], Loss: 0.0004, Accuracy: 1.0000
Test Loss: 2.9896, Test Accuracy: 0.7817
Epoch [11/20], Loss: 0.0001, Accuracy: 1.0000
Test Loss: 2.9919, Test Accuracy: 0.7919
Epoch [12/20], Loss: 0.0000, Accuracy: 1.0000
Test L

In [26]:
from transformers import AutoTokenizer, BioGptForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model_llm = BioGptForCausalLM.from_pretrained("microsoft/biogpt").to(device)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_llm.to(device)

BioGptForCausalLM(
  (biogpt): BioGptModel(
    (embed_tokens): BioGptScaledWordEmbedding(42384, 1024, padding_idx=1)
    (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-23): 24 x BioGptDecoderLayer(
        (self_attn): BioGptSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((

In [56]:
import requests

def generate_response(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False  # Set to True if you want streaming output
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json().get("response", "")
    else:
        raise Exception(f"Error generating response: {response.text}")

In [ ]:
def generate_explanation(class_label, confidence):
    label = class_label.replace("_", " ")
    if label == 'no tumor':
        return "No tumor detected."
    else:
        prompt = (
            f"Based on the MRI scan analysis, the classification result is '{label}' "
            f"with a confidence score of {confidence:.2f}. Provide a single line explanation of what "
            f"{label} condition typically indicates. Start with the text: The scan indicates that the patient has "
            f"a {label} tumor, which is a type of brain tumor that..."
        )
        return generate_response(prompt)

In [74]:
import json

def analyze_and_output_json(img_path):
    # Preprocess the image
    img = Image.open(img_path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)

    # Predict image class
    model.eval()
    with torch.no_grad():
        outputs = model(img)
        probs = torch.softmax(outputs, dim=1)
        class_idx = torch.argmax(probs, dim=1).item()
        confidence = probs[0][class_idx].item()
        class_label = categories[class_idx]

    # Generate explanation
    explanation = generate_explanation(class_label, confidence)

    # Create JSON output
    result = {
        "classification": class_label,
        "confidence": float(confidence),
        "explanation": explanation
    }
    return json.dumps(result, indent=4)

# Example usage
img_path = os.path.join("brain_tumor_dataset", "Testing", "meningioma_tumor", "image(2).jpg")
output = analyze_and_output_json(img_path)
print(output)

{
    "classification": "meningioma_tumor",
    "confidence": 1.0,
    "explanation": "The scan indicates that the patient has a meningioma tumor, which is a type of non-cancerous brain tumor arising from the meninges, the protective membranes surrounding the brain, often requiring surgical removal rather than aggressive treatment."
}
